# 如何分割 JSON 数据

这个 json 分割器 [分割](/docs/concepts/text_splitters/) json 数据，同时允许控制块大小。它会深度优先地遍历 json 数据并构建更小的 json 块。它会尝试保持嵌套的 json 对象完整，但如果需要，也会将它们分割以保持块大小在 min_chunk_size 和 max_chunk_size 之间。

如果值不是嵌套的 json，而是非常大的字符串，则该字符串不会被分割。如果您需要对块大小设置硬性限制，请考虑将此与递归文本分割器组合使用来处理这些块。有一个可选的预处理步骤来分割列表，首先将它们转换为 json（dict），然后按此方式分割。

1. 文本是如何被分割的：json 值。
2. 块大小是如何衡量的：按字符数。

In [ ]:
%pip install -qU langchain-text-splitters

首先，我们加载一些 JSON 数据：

In [1]:
import json

import requests

# This is a large nested json object and will be loaded as a python dict
json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()

## 基本用法

指定 `max_chunk_size` 来限制块大小：

In [2]:
from langchain_text_splitters import RecursiveJsonSplitter

splitter = RecursiveJsonSplitter(max_chunk_size=300)

要获取 json 块，请使用 `.split_json` 方法：

In [3]:
# Recursively split json data - If you need to access/manipulate the smaller json chunks
json_chunks = splitter.split_json(json_data=json_data)

for chunk in json_chunks[:3]:
    print(chunk)

{'openapi': '3.1.0', 'info': {'title': 'LangSmith', 'version': '0.1.0'}, 'servers': [{'url': 'https://api.smith.langchain.com', 'description': 'LangSmith API endpoint.'}]}
{'paths': {'/api/v1/sessions/{session_id}': {'get': {'tags': ['tracer-sessions'], 'summary': 'Read Tracer Session', 'description': 'Get a specific session.', 'operationId': 'read_tracer_session_api_v1_sessions__session_id__get'}}}}
{'paths': {'/api/v1/sessions/{session_id}': {'get': {'security': [{'API Key': []}, {'Tenant ID': []}, {'Bearer Auth': []}]}}}}


要获取 LangChain 的 [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) 对象，请使用 `.create_documents` 方法：

In [4]:
# The splitter can also output documents
docs = splitter.create_documents(texts=[json_data])

for doc in docs[:3]:
    print(doc)

page_content='{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "servers": [{"url": "https://api.smith.langchain.com", "description": "LangSmith API endpoint."}]}'
page_content='{"paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session.", "operationId": "read_tracer_session_api_v1_sessions__session_id__get"}}}}'
page_content='{"paths": {"/api/v1/sessions/{session_id}": {"get": {"security": [{"API Key": []}, {"Tenant ID": []}, {"Bearer Auth": []}]}}}}'


或者使用 `.split_text` 直接获取字符串内容：

In [5]:
texts = splitter.split_text(json_data=json_data)

print(texts[0])
print(texts[1])

{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "servers": [{"url": "https://api.smith.langchain.com", "description": "LangSmith API endpoint."}]}
{"paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session.", "operationId": "read_tracer_session_api_v1_sessions__session_id__get"}}}}


## 如何管理列表内容的块大小

请注意，此示例中的一个块大于指定的 `max_chunk_size` 300。查看其中一个较大的块，我们看到其中有一个列表对象：

In [6]:
print([len(text) for text in texts][:10])
print()
print(texts[3])

[171, 231, 126, 469, 210, 213, 237, 271, 191, 232]

{"paths": {"/api/v1/sessions/{session_id}": {"get": {"parameters": [{"name": "session_id", "in": "path", "required": true, "schema": {"type": "string", "format": "uuid", "title": "Session Id"}}, {"name": "include_stats", "in": "query", "required": false, "schema": {"type": "boolean", "default": false, "title": "Include Stats"}}, {"name": "accept", "in": "header", "required": false, "schema": {"anyOf": [{"type": "string"}, {"type": "null"}], "title": "Accept"}}]}}}}


默认情况下，json 分割器不会分割列表。

指定 `convert_lists=True` 以预处理 json，将列表内容转换为字典，其中列表的 `index:item` 作为 `key:val` 对：

In [7]:
texts = splitter.split_text(json_data=json_data, convert_lists=True)

我们来看看分块的大小。现在它们都低于最大值

In [8]:
print([len(text) for text in texts][:10])

[176, 236, 141, 203, 212, 221, 210, 213, 242, 291]


列表已被转换为字典，但保留了所有必要的上下文信息，即使它被拆分成多个块：

In [9]:
print(texts[1])

{"paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": {"0": "tracer-sessions"}, "summary": "Read Tracer Session", "description": "Get a specific session.", "operationId": "read_tracer_session_api_v1_sessions__session_id__get"}}}}


In [10]:
# We can also look at the documents
docs[1]

Document(page_content='{"paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session.", "operationId": "read_tracer_session_api_v1_sessions__session_id__get"}}}}')